# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [5]:
%pip install sounddevice

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pydub import AudioSegment
from pydub.playback import play
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write as write_wav
import whisper
import ollama # <-- using ollama for testing gemma 3n is downloaded and on my box so i will use that too 
import gradio as gr

In [ ]:
# Audio handling section - print statements are for added clarity 

In [27]:
inp_fs = 48000   # typical sample rate but we will grab device defaults 
inp_device = ''
inp_chnl = 0
block_size = 1024
out_fs = 48000
out_device = ''
out_chnl = 0 
recorded_frames = []
devices = sd.query_devices(device=None, kind=None)
for device in devices:
    print(type(device))
    if device['max_input_channels'] > 0 :
        inp_device = device['name']
        inp_fs = device['default_samplerate']
        inp_chnl = device['max_input_channels']
    elif device['max_output_channels'] > 0: 
        out_device = device['name']
        out_fs = device['default_samplerate']
        out_chnl = device['max_output_channels']
    for k,v in zip(device.keys(), device.values()):
        print(f'{k}: {v}')
    print(device)
out_fs=int(out_fs)
sd.default.samplerate = out_fs # setting to out samplerate not sure why but feels like the right thing to do for clear output/ playback

<class 'dict'>
name: MacBook Air Microphone
index: 0
hostapi: 0
max_input_channels: 1
max_output_channels: 0
default_low_input_latency: 0.0336875
default_low_output_latency: 0.01
default_high_input_latency: 0.043020833333333335
default_high_output_latency: 0.1
default_samplerate: 48000.0
{'name': 'MacBook Air Microphone', 'index': 0, 'hostapi': 0, 'max_input_channels': 1, 'max_output_channels': 0, 'default_low_input_latency': 0.0336875, 'default_low_output_latency': 0.01, 'default_high_input_latency': 0.043020833333333335, 'default_high_output_latency': 0.1, 'default_samplerate': 48000.0}
<class 'dict'>
name: MacBook Air Speakers
index: 1
hostapi: 0
max_input_channels: 0
max_output_channels: 2
default_low_input_latency: 0.01
default_low_output_latency: 0.024416666666666666
default_high_input_latency: 0.1
default_high_output_latency: 0.03375
default_samplerate: 48000.0
{'name': 'MacBook Air Speakers', 'index': 1, 'hostapi': 0, 'max_input_channels': 0, 'max_output_channels': 2, 'default_

In [2]:
def callback(indata, frames, time, status):
    """This function is called for every audio block."""
    if status:
        print(status)
    RECORDED_FRAMES.append(indata.copy())

In [16]:
try:
    with sd.InputStream(samplerate=48000, channels=1, blocksize=1024, callback=callback):
        print("Recording. Press Enter to stop.")
        input()  # Keep the stream active until Enter is pressed
except KeyboardInterrupt:
    print("\nRecording interrupted.")
finally:
    if RECORDED_FRAMES:
        # Concatenate recorded blocks and save to WAV
        audio_data = np.concatenate(RECORDED_FRAMES, axis=0)
        #print(type(out_fs))
        write_wav('output.wav', 48000, audio_data)
        print("Audio saved to output.wav")

Recording. Press Enter to stop.


Audio saved to output.wav


In [30]:
#playback

In [18]:
audio = AudioSegment.from_file('output.wav', format="wav")
model = whisper.load_model("small.en")
result = model.transcribe("output.wav")
print(result["text"])
play(audio)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Grill the steaks.


Input #0, wav, from '/var/folders/jf/tj0cfq397d3_v0nlyqcw_05h0000gn/T/tmppo8tuc3a.wav':
  Duration: 00:00:14.08, bitrate: 1536 kb/s
  Stream #0:0: Audio: pcm_s32le ([1][0][0][0] / 0x0001), 48000 Hz, 1 channels, s32, 1536 kb/s
  13.97 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

In [3]:
RECORDED_FRAMES = []

In [38]:
model = whisper.load_model("small.en") # i already loaded the model small.n is the small english-only version of whisper works well for my voice
def record():
    aud_file = 'request.wav'
    inp_fs = 48000   # typical sample rate but we will grab device defaults 
    inp_device = ''
    inp_chnl = 0
    block_size = 1024
    out_fs = 48000
    out_device = ''
    out_chnl = 0 
    recorded_frames = []
    def callback(indata, frames, time, status):
        """This function is called for every audio block."""
        if status:
            print(status)
        recorded_frames.append(indata.copy())
        
    devices = sd.query_devices(device=None, kind=None)
    for device in devices:
        print(type(device))
        if device['max_input_channels'] > 0 :
            inp_device = device['name']
            inp_fs = device['default_samplerate']
            inp_chnl = device['max_input_channels']
        elif device['max_output_channels'] > 0: 
            out_device = device['name']
            out_fs = device['default_samplerate']
            out_chnl = device['max_output_channels']
        for k,v in zip(device.keys(), device.values()):
            print(f'{k}: {v}')
        
    out_fs=int(out_fs)
    try:
        with sd.InputStream(samplerate=out_fs, channels=inp_chnl, blocksize=block_size, callback=callback):
            print("Recording. Press Enter to stop.")
            input()
    except KeyboardInterupt:
        print("recording stopped")
            # Concatenate recorded blocks and save to WAV
    finally:
        if recorded_frames:
            audio_data = np.concatenate(recorded_frames, axis=0)
            write_wav(aud_file, out_fs, audio_data)
            print(f"Audio saved to {aud_file}")
        else: 
            print("not sure why but its not working")    
    audio = AudioSegment.from_file(aud_file, format='wav')
    message = model.transcribe(aud_file)
    return message

In [20]:
sys_prmpt = """You are a helpful assistant. You answer any and all questions especially coding related questions."""
def chat_w_llm(message):
    messages = [{'role': 'user', 'content': message}]
    response = ''
    stream = ollama.chat('llama3.2', messages=messages, stream=True)
    for chunk in stream:
        response += chunk['message']['content']
        print(chunk['message']['content'],end='',flush=True)
    messages.append({'role': 'assistant', 'content': response})
    print(messages)
    return messages
    
        

In [27]:
def chat_fn(message, history=[]):
    history.append(chat_w_llm(message))
    print(history)
    return history[-1]
    

In [30]:
#gradio section 

with gr.Blocks() as ui:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="Enter your message or record audio") # Use a Textbox for manual input

    # Add a row for the record button and audio input
    with gr.Row():
        record_button = gr.Button("Record Audio")
        #audio_input = gr.Audio(source="microphone", type="filepath") # Use filepath to get the path to the recorded audio file

    clear = gr.Button("Clear")

    # Event listener for the record button
    record_button.click(
        fn=record,
        inputs=None,
        outputs=msg,  # Output the recording status to the message box
    )
    

    # Event listener for sending a message (either text or based on recording)
    msg.submit(
        fn=chat_fn,
        inputs=[msg,chatbot],
        outputs=[chatbot],
    )
    clear.click(lambda: None, None, chatbot, queue=False) # Clear chatbot history


ui.launch()#inbrowser=True)


* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


In [40]:
print(record())

<class 'dict'>
name: MacBook Air Microphone
index: 0
hostapi: 0
max_input_channels: 1
max_output_channels: 0
default_low_input_latency: 0.0336875
default_low_output_latency: 0.01
default_high_input_latency: 0.043020833333333335
default_high_output_latency: 0.1
default_samplerate: 48000.0
<class 'dict'>
name: MacBook Air Speakers
index: 1
hostapi: 0
max_input_channels: 0
max_output_channels: 2
default_low_input_latency: 0.01
default_low_output_latency: 0.024416666666666666
default_high_input_latency: 0.1
default_high_output_latency: 0.03375
default_samplerate: 48000.0
Recording. Press Enter to stop.


Audio saved to request.wav


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' What is your name?', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 2.0, 'text': ' What is your name?', 'tokens': [50363, 1867, 318, 534, 1438, 30, 50463], 'temperature': 0.0, 'avg_logprob': -0.4327879846096039, 'compression_ratio': 0.6923076923076923, 'no_speech_prob': 0.1053680032491684}], 'language': 'en'}


In [14]:
chat_w_llm('please write a poem about a girl named gabby')

## Gabby

Gabby, a whisper of sunshine and grace,
With eyes that hold starlight in their space.
A laugh like wind chimes, a melody bright,
She dances through shadows and fills them with light.

Her spirit, a garden, with blooms wild and free,
A tapestry woven for all eyes to see.
Kindness she offers, a gentle embrace,
A warmth that can brighten the coldest of days.

She chases her dreams with a passionate fire,
A soul full of longing, a burning desire.
Though moments may stumble, and shadows may fall,
Gabby rises stronger, embracing it all.

A dreamer, a thinker, a heart open wide,
She sees the beauty the world tries to hide.
With a smile and a spirit that shines from within,
Gabby, a wonder, where hope can begin. 

So listen closely, and you'll surely hear,
The echo of Gabby, banishing fear.
A girl of resilience, of grace and of might,
Gabby, a beacon, a beautiful light.





"## Gabby\n\nGabby, a whisper of sunshine and grace,\nWith eyes that hold starlight in their space.\nA laugh like wind chimes, a melody bright,\nShe dances through shadows and fills them with light.\n\nHer spirit, a garden, with blooms wild and free,\nA tapestry woven for all eyes to see.\nKindness she offers, a gentle embrace,\nA warmth that can brighten the coldest of days.\n\nShe chases her dreams with a passionate fire,\nA soul full of longing, a burning desire.\nThough moments may stumble, and shadows may fall,\nGabby rises stronger, embracing it all.\n\nA dreamer, a thinker, a heart open wide,\nShe sees the beauty the world tries to hide.\nWith a smile and a spirit that shines from within,\nGabby, a wonder, where hope can begin. \n\nSo listen closely, and you'll surely hear,\nThe echo of Gabby, banishing fear.\nA girl of resilience, of grace and of might,\nGabby, a beacon, a beautiful light.\n\n\n\n"

In [17]:
res = chat_w_llm('can you help me?')
print(res)

I can offer insights and tips on a range of subjects. Would you mind telling me a little more about what you'd like help with?I can offer insights and tips on a range of subjects. Would you mind telling me a little more about what you'd like help with?
